In [16]:
import spikeinterface.extractors as se
import spikeinterface as si
import pipeline_tdt as p
from pipeline_tdt import load_and_preprocess_si_recording
from pathlib import Path
import tdt

In [43]:
from pipeline_tdt import run_sorting

In [2]:
def run_pipeline(
    # paths and data
    tdt_folder_paths,
    output_dir,
    store="NNXr",
    nchans=16,
    t_end=None,
    # PRepro
    prepro_analysis_name="prepro_df",
    bad_channel_ids=None,
    artifact_frames_list=None,
    hyp_paths=None,
    hyp_artifactual_states=None,
    # Sorting
    sorting_analysis_name="ks2_5_nblocks=1_8s-batches",
    # misc
    rerun_existing=True,
    dry_run=False,
):

    # Check
    if not isinstance(tdt_folder_paths, (tuple, list)):
        raise ValueError(
            "`tdt_folder_paths` arg should be a list of existing TDT folders"
        )
    for tdt_folder in tdt_folder_paths:
        if not Path(tdt_folder).exists():
            raise FileNotFoundError(f"Could not find TDT folder `{tdt_folder}`")
    if t_end is None or not t_end:
        t_end = [None for _ in range(len(tdt_folder_paths))]
    else:
        if len(t_end) != len(tdt_folder_paths):
            raise ValueError(
                f"Expected `None` or a list of length {len(tdt_folder_paths)} for kwarg `t_end`"
            )
    if hyp_artifactual_states:
        if len(hyp_paths) != len(tdt_folder_paths):
            raise ValueError(
                f"`hyp_artifactual_states` kwarg was set: Expecting a list of length {len(tdt_folder_paths)} for kwarg `hyp_paths`"
            )
        if not all([Path(hyp_path).exists() for hyp_path in hyp_paths]):
            raise ValueError(
                f"`hyp_artifactual_states` kwarg was set: Could not find some of the hypnograms at the following paths:\n{hyp_paths}"
            )
    if hyp_paths is None or not hyp_paths:
        hyp_paths = [None for _ in range(len(tdt_folder_paths))]

    print(
        f"Running pipeline for TDT folder: {tdt_folder_paths}\n"
        f"End times for each block: {t_end}\n"
        f"Zero-ing out states `{hyp_artifactual_states}`\n"
        f"Hypno for each block: {hyp_paths}\n"
        f"Save all output at: {output_dir}"
    )

    # create output dir if it doesn't exist
    Path(output_dir).mkdir(exist_ok=True, parents=True)

    assert "NNX" in store
    if len(tdt_folder_paths) > 1 and artifact_frames_list is not None:
        # Should provide list of list if we concatenate
        raise NotImplementedError()

    # Load, preprocess each block, concatenate
    si_recordings = [
        load_and_preprocess_si_recording(
            tdt_folder,
            store=store,
            nchans=nchans,
            t_end=end_time,
            prepro_analysis_name=prepro_analysis_name,
            bad_channel_ids=bad_channel_ids,
            artifact_frames_list=artifact_frames_list,
            hyp_path=hyp_path,
            hyp_artifactual_states=hyp_artifactual_states,
            output_dir=output_dir,
        )
        for tdt_folder, end_time, hyp_path in zip(tdt_folder_paths, t_end, hyp_paths)
    ]
    
    concat_si_recording = si.core.concatenate_recordings(si_recordings)
    total_t = (
        concat_si_recording.get_num_frames()
        / concat_si_recording.get_sampling_frequency()
    )
    n_chans = concat_si_recording.get_num_channels()
    print(
        f"Concatenated N={len(si_recordings)} recordings. Total T={total_t}sec, n_chans={n_chans}"
    )
    return si_recordings, concat_si_recording
    # Run sorting
    print("\n\nRun sorting:")
    success = run_sorting(
        concat_si_recording,
        output_dir,
        sorting_analysis_name,
        rerun_existing=rerun_existing,
        dry_run=dry_run,
    )
    if not dry_run and not success:
        return False

    return True

In [3]:
tdt_folder_paths = [
            "/Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/",
            "/Volumes/opto_loc/Data/ACR_14/ACR_14-short2-iso/",]
output_dir = "/nvme/sorting/tdt/ACR_14-shortiso-NNXr/"
store='NNXr'
t_end = [0, 0]

recs_r, cat_r = run_pipeline(tdt_folder_paths, output_dir, store=store, t_end=t_end)

Running pipeline for TDT folder: ['/Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/', '/Volumes/opto_loc/Data/ACR_14/ACR_14-short2-iso/']
End times for each block: [0, 0]
Zero-ing out states `None`
Hypno for each block: [None, None]
Save all output at: /nvme/sorting/tdt/ACR_14-shortiso-NNXr/
Load TDT block at /Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/
t_start = 0.0, t_end=0
Load channel 1 from store NNXr, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXr
Load channel 2 from store NNXr, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXr
Load channel 3 from store NNXr, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXr
Load channel 4 from store NNXr, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXr
Load channel 5 from store NNXr, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXr
Load channel 6 from store NNXr, read from t=0.0s to t=642

In [45]:
output_dir = "/nvme/sorting/tdt/ACR_14-shortiso-NNXr/"
success = run_sorting(
        cat_r,
        output_dir,
        "ks2_5_nblocks=1_8s-batches",
        rerun_existing=True,
        dry_run=False,
    )

Saving sorting output at /nvme/sorting/tdt/ACR_14-shortiso-NNXr/ks2_5_nblocks=1_8s-batches
Running...Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /Volumes/scratch/neuropixels/matlab/external/Kilosort-wisc-v2.5
RUNNING SHELL SCRIPT: /tmp/tmp_shellscript557swag0/script.sh


write_binary_recording: 100%|##########| 9760/9760 [00:11<00:00, 879.75it/s]


RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptdcxtyybd/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptrrk19s1v/script.sh
RUNNING SHELL SCRIPT: /nvme/sorting/tdt/ACR_14-shortiso-NNXr/ks2_5_nblocks=1_8s-batches/run_kilosort2_5.sh
Set Matlab shell to /usr/bin/bash



fatal: detected dubious ownership in repository at '/Volumes/scratch/neuropixels/matlab/external/Kilosort-wisc-v2.5'
To add an exception for this directory, call:

	git config --global --add safe.directory /Volumes/scratch/neuropixels/matlab/external/Kilosort-wisc-v2.5




                            < M A T L A B (R) >

                  Copyright 1984-2019 The MathWorks, Inc.

                  R2019b (9.7.0.1190202) 64-bit (glnxa64)

                              August 21, 2019



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Setting latest mkl version

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time   4s. Loading raw data and applying filters... 

Time  72s. Finished preprocessing 909 batches. 

Drift correction ENABLED

pitch is 50 um

0.09 sec, 1 batches, 4714 spikes 

2.37 sec, 101 batches, 455764 spikes 

4.62 sec, 201 batches, 880437 spikes 

7.02 sec, 301 batches, 1502408 spikes 

9.59 sec, 401 batches, 2261209 spikes 

12.04 sec, 501 batches, 3004213 spikes 

14.34 sec, 601 batches, 3550125 spikes 

16.71 sec, 701 batches, 4004568 spikes 

18.98 sec, 801 batches, 4447615 spikes 

21.26 sec, 901 batches, 4877007 spikes 

21.44 sec,

In [4]:
tdt_folder_paths = [
            "/Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/",
            "/Volumes/opto_loc/Data/ACR_14/ACR_14-short2-iso/",]
output_dir = "/nvme/sorting/tdt/ACR_14-shortiso-NNXo/"
store='NNXo'
t_end = [0, 0]

recs_o, cat_o = run_pipeline(tdt_folder_paths, output_dir, store=store, t_end=t_end)

Running pipeline for TDT folder: ['/Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/', '/Volumes/opto_loc/Data/ACR_14/ACR_14-short2-iso/']
End times for each block: [0, 0]
Zero-ing out states `None`
Hypno for each block: [None, None]
Save all output at: /nvme/sorting/tdt/ACR_14-shortiso-NNXo/
Load TDT block at /Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/
t_start = 0.0, t_end=0
Load channel 1 from store NNXo, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Load channel 2 from store NNXo, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Load channel 3 from store NNXo, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Load channel 4 from store NNXo, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Load channel 5 from store NNXo, read from t=0.0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Load channel 6 from store NNXo, read from t=0.0s to t=642

In [46]:
output_dir = "/nvme/sorting/tdt/ACR_14-shortiso-NNXo/"
success = run_sorting(
        cat_o,
        output_dir,
        "ks2_5_nblocks=1_8s-batches",
        rerun_existing=True,
        dry_run=False,
    )

Saving sorting output at /nvme/sorting/tdt/ACR_14-shortiso-NNXo/ks2_5_nblocks=1_8s-batches
Running...Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /Volumes/scratch/neuropixels/matlab/external/Kilosort-wisc-v2.5
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptr937sk16/script.sh


write_binary_recording: 100%|##########| 9760/9760 [00:11<00:00, 884.99it/s]


RUNNING SHELL SCRIPT: /tmp/tmp_shellscript16go124m/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscript0lf49zj5/script.sh
RUNNING SHELL SCRIPT: /nvme/sorting/tdt/ACR_14-shortiso-NNXo/ks2_5_nblocks=1_8s-batches/run_kilosort2_5.sh
Set Matlab shell to /usr/bin/bash



fatal: detected dubious ownership in repository at '/Volumes/scratch/neuropixels/matlab/external/Kilosort-wisc-v2.5'
To add an exception for this directory, call:

	git config --global --add safe.directory /Volumes/scratch/neuropixels/matlab/external/Kilosort-wisc-v2.5




                            < M A T L A B (R) >

                  Copyright 1984-2019 The MathWorks, Inc.

                  R2019b (9.7.0.1190202) 64-bit (glnxa64)

                              August 21, 2019



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Setting latest mkl version

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time   4s. Loading raw data and applying filters... 

Time  73s. Finished preprocessing 909 batches. 

Drift correction ENABLED

pitch is 50 um

0.07 sec, 1 batches, 2329 spikes 

2.37 sec, 101 batches, 216821 spikes 

4.67 sec, 201 batches, 418725 spikes 

7.01 sec, 301 batches, 657128 spikes 

9.33 sec, 401 batches, 922731 spikes 

11.72 sec, 501 batches, 1215419 spikes 

14.05 sec, 601 batches, 1460089 spikes 

16.36 sec, 701 batches, 1688925 spikes 

18.69 sec, 801 batches, 1910523 spikes 

21.10 sec, 901 batches, 2123662 spikes 

21.28 sec, 9

In [8]:
cat_r.get_num_frames()
cat_o.get_num_frames()
cat_r.get_num_samples()
cat_o.get_num_samples()

238276610

In [48]:
sorto = se.KiloSortSortingExtractor("/nvme/sorting/tdt/ACR_14-shortiso-NNXo/ks2_5_nblocks=1_8s-batches/")
sortp = se.KiloSortSortingExtractor("/nvme/sorting/tdt/ACR_14-shortiso-NNXr/ks2_5_nblocks=1_8s-batches/")

In [54]:
(sorto.get_all_spike_trains()[0][0]/24414.06)[-1]

9759.795052523014

In [55]:
(sortp.get_all_spike_trains()[0][0]/24414.06)[-1]

9759.810371564581

In [17]:
p1 = '/Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/'
p2 = '/Volumes/opto_loc/Data/ACR_14/ACR_14-short2-iso/'

d1 = tdt.read_block(p1, store=['NNXr', 'NNXo'], channel=[2, 16])
d2 = tdt.read_block(p2, store=['NNXr', 'NNXo'], channel=[2, 16])

read from t=0s to t=6426.9s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Using 24414.0625 Hz as SEV sampling rate for NNXr
read from t=0s to t=3333.13s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Using 24414.0625 Hz as SEV sampling rate for NNXr


In [68]:
p2 = '/Volumes/opto_loc/Data/ACR_14/ACR_14-short2-iso/'

dd1 = tdt.read_block(p2, store=['NNXr', 'NNXo'], channel=[1], t1=0, t2=1)

read from t=0s to t=950.14s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Using 24414.0625 Hz as SEV sampling rate for NNXr


In [69]:
len(dd1.streams.NNXo.data)

24415

In [20]:
d1.info

tankpath:	'/Volumes/opto_loc/Data/ACR_14'
blockname:	'ACR_14-short1-iso'
start_date:	datetime.datetime(2022, 9, 30, 12, 8, 22, 999999)
utc_start_time:	'12:08:22'
stop_date:	datetime.datetime(2022, 9, 30, 13, 55, 29, 901070)
utc_stop_time:	'13:55:29'
duration:	datetime.timedelta(seconds=6426, microseconds=901071)
stream_channel:	[2, 16]
snip_channel:	[2, 16]

In [21]:
d2.info

tankpath:	'/Volumes/opto_loc/Data/ACR_14'
blockname:	'ACR_14-short2-iso'
start_date:	datetime.datetime(2022, 9, 30, 13, 55, 38, 999999)
utc_start_time:	'13:55:38'
stop_date:	datetime.datetime(2022, 9, 30, 14, 51, 12, 130175)
utc_stop_time:	'14:51:12'
duration:	datetime.timedelta(seconds=3333, microseconds=130176)
stream_channel:	[2, 16]
snip_channel:	[2, 16]

In [39]:
d1.streams.NNXr

channels:	[2, 16]
data:	array([[-4.4031996e-05, -3.8015998e-05, -4.9279995e-05, ...,
         1.8220799e-04,  1.9417598e-04,  0.0000000e+00],
       [-2.3039998e-05, -3.3407996e-05, -2.7199998e-05, ...,
         9.2223997e-04,  9.1846392e-04,  0.0000000e+00]], dtype=float32)
name:	'NNXr'
fs:	24414.0625
start_time:	0.0

In [67]:
d2.streams.NNXr.data[0]

(81371137,)

In [38]:
156905473/d1.streams.NNXr.fs

6426.84817408

In [29]:
d1.streams.NNXo.data[0].shape

(156905473,)

In [64]:
subject = 'ACR_14'

In [70]:
op = '/Volumes/opto_loc/Data/ACR_14/sorting_data/sdpi-NNXo/ks2_5_nblocks=1_8s-batches/'
pp = '/Volumes/opto_loc/Data/ACR_14/sorting_data/sdpi-NNXr/ks2_5_nblocks=1_8s-batches/'
o = se.KiloSortSortingExtractor(op)
p = se.KiloSortSortingExtractor(pp)
ot = (o.get_all_spike_trains()[0][0]/24414.06)[-1]
pt = (p.get_all_spike_trains()[0][0]/24414.06)[-1]

print('Final Spike Time, Probe-1', pt)
print('Final Spike Time, Probe-2 (optrode):', ot)

Final Spike Time, Probe-1 257066.63221930314
Final Spike Time, Probe-2 (optrode): 257066.62816426271


In [85]:
'theoretical length of concatenated recording: '+str(86495+85777.12390144+84794.5806)

'theoretical length of concatenated recording: 257066.70450144'

In [71]:
op = '/Volumes/opto_loc/Data/ACR_14/sorting_data/laser1-NNXo/ks2_5_nblocks=1_8s-batches/'
pp = '/Volumes/opto_loc/Data/ACR_14/sorting_data/laser1-NNXr/ks2_5_nblocks=1_8s-batches/'
o = se.KiloSortSortingExtractor(op)
p = se.KiloSortSortingExtractor(pp)
ot = (o.get_all_spike_trains()[0][0]/24414.06)[-1]
pt = (p.get_all_spike_trains()[0][0]/24414.06)[-1]

print('Final Spike Time, Probe-1', pt)
print('Final Spike Time, Probe-2 (optrode):', ot)

Final Spike Time, Probe-1 130886.94260602292
Final Spike Time, Probe-2 (optrode): 130897.00901857372


In [84]:
'theoretical length of concatenated recording: '+str(37450+50151.29296896+43295.70308096) 

'theoretical length of concatenated recording: 130896.99604992'

In [72]:
op = '/Volumes/opto_loc/Data/ACR_14/sorting_data/swi-NNXo/ks2_5_nblocks=1_8s-batches/'
pp = '/Volumes/opto_loc/Data/ACR_14/sorting_data/swi-NNXr/ks2_5_nblocks=1_8s-batches/'
o = se.KiloSortSortingExtractor(op)
p = se.KiloSortSortingExtractor(pp)
ot = (o.get_all_spike_trains()[0][0]/24414.06)[-1]
pt = (p.get_all_spike_trains()[0][0]/24414.06)[-1]

print('Final Spike Time, Probe-1', pt)
print('Final Spike Time, Probe-2 (optrode):', ot)

Final Spike Time, Probe-1 177735.244199449
Final Spike Time, Probe-2 (optrode): 177725.08972288918


In [83]:
'theoretical length of concatenated recording: '+str(73385.907709+86464.616425+17884.876654)

'theoretical length of concatenated recording: 177735.400788'

In [86]:
path = '/Volumes/opto_loc/Data/ACR_14/ACR_14-laser1/'
laser1 = tdt.read_block(path, store=['NNXr', 'NNXo'], channel=[1], t1=0, t2=1)

read from t=0s to t=856.79s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Using 24414.0625 Hz as SEV sampling rate for NNXr


In [90]:
path = '/Volumes/opto_loc/Data/ACR_14/ACR_14-swi/'
swi = tdt.read_block(path, store=['NNXr', 'NNXo'], channel=[1], t1=0, t2=1)

read from t=0s to t=791.83s
Using 24414.0625 Hz as SEV sampling rate for NNXo
Using 24414.0625 Hz as SEV sampling rate for NNXr
